In [9]:
import torch
def find_corners_index(coords):
        corners = []  #index of 4 corners
        center = torch.mean(coords,dim=0)
        for i in range(9):
            if torch.norm(coords[i,:] - center) < 1e-6:
                continue
            corners.append(i)
        selected_corners = [corners[0]] #4 selected corners to represent the lattice
        for i in range(1,8):
            for j in range(1,8):
                for k in range(1,8):
                    if torch.norm((coords[corners[0],:]-coords[corners[i],:]-coords[corners[j],:]+coords[corners[k],:])) < 1e-6:
                        if torch.norm((coords[corners[0],:]+coords[corners[i],:]+coords[corners[j],:]+coords[corners[k],:])-4*center) > 1e-6:
                            for h in range(1,8):
                                if torch.norm((coords[corners[k],:]+coords[corners[h],:])-2*center) < 1e-6:
                                    return corners, selected_corners + [corners[i],corners[j],corners[h]]
        return corners, [0,1,2,3]
def shrink_coords(ini_coords,batch_size):
        reshaped_coords = ini_coords.view(batch_size,-1,3)
        num_node = reshaped_coords.shape[1]
        result = torch.zeros((batch_size,5,3))
        for i in range(batch_size):
            corners, selected_corners = find_corners_index(reshaped_coords[i,:,:])
            print(corners)
            print(selected_corners)
            for j in range(4):
                result[i,j,:] = reshaped_coords[i,selected_corners[j],:]
            position = 4
            for j in range(num_node):
                if j not in corners:
                    result[i,position,:] = reshaped_coords[i,j,:]
                    position += 1
        return result
a = torch.tensor([[[1,1,1],[0,0,0],[1,0,0],[1,0,1],[1,1,0],[0.5,0.5,0.5],[0,0,1],[0,1,0],[0,1,1]]])
b = shrink_coords(a,1)
b

[0, 1, 2, 3, 4, 6, 7, 8]
[0, 3, 4, 8]


tensor([[[1.0000, 1.0000, 1.0000],
         [1.0000, 0.0000, 1.0000],
         [1.0000, 1.0000, 0.0000],
         [0.0000, 1.0000, 1.0000],
         [0.5000, 0.5000, 0.5000]]])

In [10]:
import numpy as np
a = np.array([[ 0.1282, -0.0223,  0.1897],
        [ 0.1367, -0.0853,  0.1490],
        [-0.1878,  0.0074, -0.0320],
        [ 0.1793,  0.0556,  0.0727],
        [ 0.0000,  0.0000,  0.0000]])
c = a[1] + a[2] - a[0] + a[3] - a[0]
c + a[0]

array([-2.77555756e-17,  0.00000000e+00, -2.77555756e-17])